Librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import pickle
import psutil

Carga de datos

In [2]:
clientes = pd.read_csv('cliente_tabla.csv')
productos = pd.read_csv('producto_tabla.csv')
#sample_submission = pd.read_csv('sample_submission.csv')
#test  = pd.read_csv('test.csv')
town_state  = pd.read_csv('town_state.csv')

In [3]:
train = pd.read_csv('train.csv')

Para hacer seguimiento a la memoria

In [4]:
def variables_por_tamaño():
    variables = {k: sys.getsizeof(v) for k, v in globals().items() if not k.startswith('_')}
    
    variables_en_gb = {k: v / 1024**3 for k, v in variables.items()}
    
    tipos_variables = {k: type(v).__name__ for k, v in globals().items() if not k.startswith('_')}
    
    df = pd.DataFrame({'Variable': list(variables_en_gb.keys()),
                       'Tipo': list(tipos_variables.values()),
                       'Tamaño_GB': list(variables_en_gb.values())})
    
    df = df.sort_values('Tamaño_GB', ascending=False).reset_index(drop=True)
    
    # Mostrar la memoria disponible
    memoria_disponible()
    
    return df



def Gb_por_variable(variable):
    print(sys.getsizeof(variable)/(1024**3))
    memoria_disponible()

def memoria_disponible():
    mem = psutil.virtual_memory()
    mem_total_gb = mem.total / (1024 ** 3)  # Memoria total en GB
    mem_disponible_gb = mem.available / (1024 ** 3)  # Memoria disponible en GB

    print("Memoria Total:", mem_total_gb, "GB")
    print("Memoria Disponible:", mem_disponible_gb, "GB")


In [10]:
train.columns

Index(['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID',
       'Producto_ID', 'Venta_uni_hoy', 'Venta_hoy', 'Dev_uni_proxima',
       'Dev_proxima', 'Demanda_uni_equil'],
      dtype='object')

In [6]:
df_ventas_pivot = pd.pivot_table(data=train,
                                 index=['Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID'],
                                 columns='Semana',
                                 values='Venta_uni_hoy').fillna(0)

nombre_archivo = 'Pkl/df_ventas_pivot.pkl'
with open(nombre_archivo, 'wb') as archivo_pkl:
    pickle.dump(df_ventas_pivot, archivo_pkl)

In [12]:
df_devoluciones_pivot = pd.pivot_table(data=train,
                                 index=['Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID'],
                                 columns='Semana',
                                 values='Dev_uni_proxima').fillna(0) 

nombre_archivo = 'Pkl/df_devoluciones_pivot.pkl'
with open(nombre_archivo, 'wb') as archivo_pkl:
    pickle.dump(df_devoluciones_pivot, archivo_pkl)

In [131]:
df_ventas_pivot.head(1+3)

Semana                                                 3    4    5    6    7  \
Agencia_ID Canal_ID Ruta_SAK Cliente_ID Producto_ID                            
1110       7        3301     15766      325          0.0  1.0  0.0  0.0  0.0   
                                        328          0.0  1.0  0.0  0.0  0.0   
                                        1212         3.0  4.0  5.0  0.0  0.0   
                                        1216         4.0  2.0  3.0  1.0  2.0   

Semana                                                 8    9  
Agencia_ID Canal_ID Ruta_SAK Cliente_ID Producto_ID            
1110       7        3301     15766      325          0.0  0.0  
                                        328          0.0  0.0  
                                        1212         4.0  1.0  
                                        1216         5.0  0.0

In [192]:
np.arange(max(df_ventas_pivot.columns)+1,max(df_ventas_pivot.columns)+3)

array([10, 11])

In [219]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import LinearRegression


y_true = np.array([10, 5, 11, 4, 12, 4])

def rf_pred(y_true,periodos):
    y_series = pd.Series(y_true)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    forecaster_rf = ForecasterAutoreg(regressor=model,lags=periodos)
    forecaster_rf.fit(y=y_series)
    y_pred = forecaster_rf.predict(steps=periodos)
    return np.array([y_pred.iloc[0],y_pred.iloc[1]])

def rl_pred(y_true, periodos):
    y_series = np.array(y_true
    model = LinearRegression()
    periodos_pasados = np.arange(1, len(y_true)+1).reshape(-1, 1)
    model.fit(periodos_pasados, y_series)
    periodos_futuros = np.arange(len(y_true) + 1, len(y_true) + periodos + 1).reshape(-1, 1)
    y_pred = model.predict(periodos_futuros)
    return y_pred


def revisar_linealidad(y_true,periodos):
    cof_coerr = np.corrcoef(y_true,df_ventas_pivot.columns)[0][1]
    print(cof_coerr)
    if cof_coerr > 0.75:
        return rl_pred(y_true,periodos)
    else:
        return rf_pred(y_true,periodos)
    
y_true=df_ventas_pivot.iloc[4]
y_true = np.array([10, 5, 11, 15, 30, 10,19])
revisar_linealidad(y_true,2)

0.526896950646487


array([22.11, 21.14])

In [130]:
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import warnings


warnings.simplefilter("ignore")

pes = range(3)
qes = range(3)

y_true = np.array([10, 5, 11, 4, 12, 4])

print(y_true.mean())
# Ajustar el modelo ARIMA y calcular MSE intramuestra
for p in pes:
    for q in qes:
        try:
            modelo = ARIMA(y_true, order=(p, 0, q)).fit()
            y_pred = modelo.predict(start=0, end=len(y_true)-1,warnings=False)
            mse = mean_squared_error(y_true, y_pred)            
            print("p =", p, "q =", q, "MSE =", mse)
            print("Modelo ARIMA(", p, ", 0,", q, ") - Parámetros:", modelo.params)
            print("Predicciones:", y_pred)
            print("-------------------------")
            print(np.concatenate([[y_true[0]], np.cumsum(y_pred)]))
        except:
            print("No se pudo ajustar ARIMA(", p, ", 0,", q, ")")


7.666666666666667
p = 0 q = 0 MSE = 11.555555555578627
Modelo ARIMA( 0 , 0, 0 ) - Parámetros: [ 7.66666186 11.55552006]
Predicciones: [7.66666186 7.66666186 7.66666186 7.66666186 7.66666186 7.66666186]
-------------------------
[10.          7.66666186 15.33332373 22.99998559 30.66664745 38.33330932
 45.99997118]
p = 0 q = 1 MSE = 4.342740015582335
Modelo ARIMA( 0 , 0, 1 ) - Parámetros: [ 7.74995059 -0.99874014  3.18819904]
Predicciones: [7.74995059 6.62492678 8.83323368 6.12487887 9.44984963 5.62483148]
-------------------------
[10.          7.74995059 14.37487738 23.20811106 29.33298992 38.78283955
 44.40767103]
p = 0 q = 2 MSE = 2.172595053032902
Modelo ARIMA( 0 , 0, 2 ) - Parámetros: [ 7.77812197 -1.71768164  0.99971527  0.99586478]
Predicciones: [ 7.77812197  6.2362864   9.54739072  5.40734068 10.60320466  4.72712142]
-------------------------
[10.          7.77812197 14.01440837 23.56179909 28.96913977 39.57234443
 44.29946586]
p = 1 q = 0 MSE = 1.0259163854338147
Modelo ARIMA( 